## Minesweeper

Group 1

Class: FCS7

Team Members: Tee Wei Ping, Sih Jia Qi, Kritchanat


## Minesweeper Game Introduction:
Minesweeper is a classic single-player puzzle game that challenges players to uncover hidden mines on a grid-based board. The objective of the game is to clear the board without detonating any mines. Players use logical deduction and strategy to reveal safe cells while avoiding mines.

## Type of Agent
The agent to be modelled is a Utility-Based Agent. Its primary objective is to minimise the number of moves required to complete the game or maximise the probability of successfully uncovering safe cells while avoiding mines.

## Type of Environment
| Properties | Elaborations |
| --- | --- |
| Not Accessible | All board information including the current and adjacent grids is hidden from the agent. The initial nor the goal states are given to the agent. |
| Deterministic | The next state of the environment is completely determined by the user. All mines are also randomly assigned in different locations. |
| Episodic | The current action taken will influence the next actions. When a cell is visited, the agent will not be able to revisit the same location as it will result in the same outcome. |
| Static | Environment does not change while considering the next step. |
| Discrete | All movements and positions are in discrete domains. There is a limited number of distinct precepts and actions. In Minesweeper, agents can move north, south, east, or west one step at a time. These steps are considered finite and there are no options for partial steps. |

## Problem Formulation
| Properties | Description |
| --- | --- |
| Definition of Problem | Given a partially observable environment with hidden mines, players are to make use of a set of actions to achieve the goal of uncovering all safe squares without hitting a mine. This challenge requires players to make optimal decisions based on limited information to reach the desired outcome in the most efficient manner |
| Initial State | The initial state of the environment is a grid with hidden hazards randomly distributed. The exact locations of these mines are unknown to the agent. Additionally, all cells are initially covered, and the agent's position is not specified. |
| Action Sets | Reveal: The agent can choose to reveal a cell on the grid. This action uncovers the content of the selected cell, revealing whether it contains a mine or a safe tile. Flag: The agent can choose to flag a cell suspected of containing a mine. This action helps the agent keep track of potential mine locations and avoid them in subsequent actions. |
| Goal Test Predicate | The goal is achieved when: 1. All safe tiles on the grid are revealed, and no mines are detonated. 2. The agent takes the least number of actions/ steps to reach the goal. |
| Cost Function | Each movement taken by the agent will increase the number of actions taken to reach the goal by 1 |
| Solution | All cells are revealed and no mines are detonated, which indicates successful completion of the game. |

## Gameplay Overview:

`Board Setup`: The game board consists of a grid of squares, each of which may contain either a mine or an empty space. At the start of the game, all squares are covered.

`Objective`: The primary goal is to uncover all the safe cells on the board without triggering any mines. Players must use clues provided by revealed cells to identify and avoid mines.

`Revealing Cells`: Players can click or select cells to reveal them. Upon revealing a cell, several outcomes are possible:

- If the revealed cell contains a mine, the game ends, and the player loses.
- If the revealed cell is empty, it will display a number indicating the number of adjacent cells containing mines.
- If the revealed cell has no adjacent mines, it will reveal adjacent cells automatically, continuing until cells with adjacent mines are encountered.

`Using Clues`: Players use the numbers revealed on cells to deduce the locations of mines. The numbers indicate how many mines are adjacent to that cell. By analyzing these clues, players can strategically mark potential mine locations and safely uncover other cells.

`Marking Mines`: To assist in gameplay, players can mark cells they suspect contain mines with a flag or question mark. This helps keep track of potential mine locations and prevents accidental clicks on flagged cells.

`Game Completion`: The game is won when all safe cells are successfully uncovered, revealing the entire board without detonating any mines.

Minesweeper combines elements of logic, deduction, and strategy to provide a challenging and rewarding gaming experience. With careful observation and smart decision-making, players can successfully navigate the minefield and clear the board.

#### Note: In this notebook, the game will not be interactive. It will only showcase the various search algorithms used to achieve the aim of this game with the least number of steps

In [16]:
# imports
import numpy as np
import random
import heapq
from collections import deque

import pygame

# Define constants
CELL_SIZE = 30
WHITE = (255, 255, 255)
GRAY = (192, 192, 192)
BLACK = (0, 0, 0)

### Algorithms used
1) Breadth-First Search (BFS)
- Explores level by level, starting from initial cell
- Uses a queue to keep track of cells to be visited next
- Visits all adjacent cells iteratively until there are no more cells to explore.
- Cells are marked as visited to avoid revisiting them.
- Systematically explores cells without getting trapped in loops and ensures that cells closest to starting point are explored first

2) Uniform-Cost Search (UCS)
- Similar to BFS but takes into account the cost of each step
- Uses a priority queue to prioritize cells with lowest cost 
- The cost is calculated based on the distance from the starting cell.''
- Explored based on their costs, ensuring that cells with lower costs are visited first.
- Aims to find the least-cost path to reveal safe cells.

3) Depth-First Search (DFS)
- Recursively explores adjacent cells in a depth-first manner till it reaches a dead-end or visits all cells before backtracking
- May not always find the shortest path to reveal safe cells, but explores deeper into a branch before backtracking.

4) Iterative Deepening Search (IDS)
- Performs DFS & BFS with increasing depth limits till goal is found
- Ensures completeness and optimality while avoiding the exponential space complexity of BFS
- Beneficial as it is a good balance between BFS & DFS, gradually increasing depth to uncover safer cells

5) Depth-Limited Search (DLS) 
- A variant of depth-first search (DFS) where the maximum depth of the search tree is limited to a predetermined value. 
- It explores a path in the search tree up to a certain depth before backtracking to explore other paths.
- DLS is suitable for scenarios where the search space is large or infinite.

6) A Star Search (A*Search)
- Uses a heuristic function (h) along with the cost function (g) to estimate the cost of reaching the goal from the current cell.
- Explores each cell based on a combination of the cost to reach the cell from the starting point and an estimate of the cost to reach the goal.

7) Greedy Search
- Similar to A* search, it uses a heuristic function (h) to prioritize cells that are the most promising to explore.
- Makes decision based on information available at the current step
- Does not consider the accumulated cost from the starting cell, making it less optimal compared to A* search.
   

In [48]:
class Minesweeper:
    def __init__(self, size=10, mines=10):
        self.size = size
        self.mines = mines
        self.board = np.zeros((size, size), dtype=int)  # 0 represents unrevealed cell
        self.visited = np.zeros((size, size), dtype=bool)
        self.generate_board()

    def generate_board(self):
        # Randomly place mines on the board
        mine_indices = random.sample(range(self.size * self.size), self.mines)
        for idx in mine_indices:
            row = idx // self.size
            col = idx % self.size
            self.board[row][col] = -1  # -1 represents a mine
        # Calculate the numbers in cells adjacent to mines
        for r in range(self.size):
            for c in range(self.size):
                if self.board[r][c] == -1:
                    continue
                count = 0
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if (dr != 0 or dc != 0) and 0 <= r + dr < self.size and 0 <= c + dc < self.size and self.board[r + dr][c + dc] == -1:
                            count += 1
                self.board[r][c] = count

    def print_board(self, reveal=False, path=None):
        cell_width = 4  # Adjust the width of each cell as needed
        print("  ", end="")
        for col in range(self.size):
            print(str(col).rjust(cell_width), end=" ")
        print("\n  +" + "-" * (cell_width + 1) * self.size + "+")
        for row in range(self.size):
            print(str(row).rjust(2) + " |", end="")
            for col in range(self.size):
                if not reveal and not self.visited[row][col]:
                    print("#".rjust(cell_width), end="|")  # Unrevealed cell
                else:
                    if path and self.visited[row][col]:
                        print("\033[1;34;40m" + str(self.board[row][col]).rjust(cell_width) + "\033[1;37;40m", end="|")  # Visited value with blue color
                    elif self.board[row][col] == -1:
                        print("*".rjust(cell_width), end="|")  # Mine
                    elif self.board[row][col] == 0:
                        print(".".rjust(cell_width), end="|")  # Empty cell
                    else:
                        print(str(self.board[row][col]).rjust(cell_width), end="|")  # Numbered cell
            print("\n  +" + "-" * (cell_width + 1) * self.size + "+")
    
    # 1. breadth-first search
    def bfs(self, row, col):
            queue = deque([(row, col)])
            while queue:
                r, c = queue.popleft()
                if not (0 <= r < self.size and 0 <= c < self.size) or self.visited[r][c]:
                    continue
                self.visited[r][c] = True
                if self.board[r][c] == 0:
                    for dr in [-1, 0, 1]:
                        for dc in [-1, 0, 1]:
                            queue.append((r + dr, c + dc))
    
    # 2. uniform-cost search
    def ucs(self, row, col):
        start = (row, col)
        queue = [(0, start)]  # Priority queue with initial cost 0
        while queue:
            cost, node = heapq.heappop(queue)
            if self.board[node[0]][node[1]] == -1:
                continue  # Skip if it's a mine
            if not self.visited[node[0]][node[1]]:
                self.visited[node[0]][node[1]] = True
                if self.board[node[0]][node[1]] == 0:
                    for dr in [-1, 0, 1]:
                        for dc in [-1, 0, 1]:
                            new_node = (node[0] + dr, node[1] + dc)
                            if 0 <= new_node[0] < self.size and 0 <= new_node[1] < self.size:
                                heapq.heappush(queue, (cost + 1, new_node))
            if self.board[node[0]][node[1]] > 0:
                return node  # Found a safe cell
        return None  # No safe cell found
    
    # 3. depth-first search
    def dfs(self, row, col):
        if row < 0 or row >= self.size or col < 0 or col >= self.size or self.visited[row][col]:
            return
        self.visited[row][col] = True
        if self.board[row][col] == 0:
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    self.dfs(row + dr, col + dc)
        
    # 4. iterative deepening search
    def ids(self, row, col):
        start = (row, col)
        depth_limit = 5
        while True:
            self.visited = np.zeros((self.size, self.size), dtype=bool)
            result = self.depth_limited_search(start, depth_limit)
            if result:
                return result
            depth_limit += 1

    # 5. depth-limited search  
    def depth_limited_search(self, start, depth_limit=1):
        return self.recursive_dls(start, depth_limit)

    def recursive_dls(self, node, depth_limit):
        if depth_limit == 0:
            return None
        if self.board[node[0]][node[1]] == -1:
            return None
        if self.board[node[0]][node[1]] > 0:
            self.visited[node[0]][node[1]] = True
            return node
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                child = (node[0] + dr, node[1] + dc)
                if 0 <= child[0] < self.size and 0 <= child[1] < self.size and not self.visited[child[0]][child[1]]:
                    result = self.recursive_dls(child, depth_limit - 1)
                    if result:
                        return result
        return None
    
    # 6. A* search
    def a_star(self, start):
        h = lambda node: abs(start[0] - node[0]) + abs(start[1] - node[1])
        g = 1  # Cost of each step
        queue = [(h(start), g, start)]
        while queue:
            _, g, (r, c) = heapq.heappop(queue)
            if not (0 <= r < self.size and 0 <= c < self.size) or self.visited[r][c]:
                continue
            self.visited[r][c] = True
            if self.board[r][c] == 0:
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        heapq.heappush(queue, (h((r + dr, c + dc)) + g, g + 1, (r + dr, c + dc)))

    # 7. greedy search                 
    def greedy(self, start):
        h = lambda node: abs(start[0] - node[0]) + abs(start[1] - node[1])
        queue = [(h(start), start)]
        while queue:
            _, (r, c) = heapq.heappop(queue)
            if not (0 <= r < self.size and 0 <= c < self.size) or self.visited[r][c]:
                continue
            self.visited[r][c] = True
            if self.board[r][c] == 0:
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if 0 <= r + dr < self.size and 0 <= c + dc < self.size:
                            heapq.heappush(queue, (h((r + dr, c + dc)), (r + dr, c + dc)))
                            
    def count_moves(self, algorithm):
        moves = 0
        for r in range(self.size):
            for c in range(self.size):
                if not self.visited[r][c] and self.board[r][c] != -1:
                    moves += 1
                    # depth-first search
                    if algorithm == "dfs":
                        self.dfs(r, c)
                    # depth-limited search
                    elif algorithm == "dls": 
                        self.depth_limited_search((r, c))
                    # uniform-cost search
                    elif algorithm == "ucs":
                        self.ucs(r, c)
                    # breadth-first search
                    elif algorithm == "bfs":
                        self.bfs(r, c)
                     # iterative deepening search
                    elif algorithm == "ids":
                        self.ids(r, c)
                    elif algorithm == "a_star":
                        self.a_star((r, c))
                    elif algorithm == "greedy":
                        self.greedy((r, c))
        return moves

In [54]:
game = Minesweeper(size=5, mines=5)
print("Minesweeper Board:")
game.print_board(reveal=True)

print("\nNumber of moves using DFS:", game.count_moves("dfs"))
game = Minesweeper(size=5, mines=5)
game.print_board(reveal=True)

print("\nNumber of moves using Uniform-cost Search:", game.count_moves("ucs"))
game = Minesweeper(size=5, mines=5)
game.print_board(reveal=True)

print("\nNumber of moves using BFS:", game.count_moves("bfs"))
game = Minesweeper(size=5, mines=5)
game.print_board(reveal=True)

print("\nNumber of moves using Depth-Limited Search:", game.count_moves("dls"))
game = Minesweeper(size=5, mines=5)
game.print_board(reveal=True)

print("\nNumber of moves using Iterative Deepening Search:", game.count_moves("ids"))
game = Minesweeper(size=5, mines=5)
game.print_board(reveal=True)

print("\nNumber of moves using A*:", game.count_moves("a_star"))
game = Minesweeper(size=5, mines=5)
game.print_board(reveal=True)

print("\nNumber of moves using Greedy:", game.count_moves("greedy"))
game = Minesweeper(size=5, mines=5)
game.print_board(reveal=True)


Minesweeper Board:
     0    1    2    3    4 
  +-------------------------+
 0 |   .|   .|   1|   *|   1|
  +-------------------------+
 1 |   .|   .|   1|   2|   2|
  +-------------------------+
 2 |   1|   1|   .|   2|   *|
  +-------------------------+
 3 |   *|   1|   .|   3|   *|
  +-------------------------+
 4 |   1|   1|   .|   2|   *|
  +-------------------------+

Number of moves using DFS: 4
     0    1    2    3    4 
  +-------------------------+
 0 |   .|   .|   1|   1|   1|
  +-------------------------+
 1 |   .|   1|   2|   *|   1|
  +-------------------------+
 2 |   .|   1|   *|   3|   2|
  +-------------------------+
 3 |   2|   3|   3|   *|   1|
  +-------------------------+
 4 |   *|   *|   2|   1|   1|
  +-------------------------+

Number of moves using Uniform-cost Search: 17
     0    1    2    3    4 
  +-------------------------+
 0 |   .|   1|   *|   1|   .|
  +-------------------------+
 1 |   .|   1|   2|   2|   1|
  +-------------------------+
 2 |   .| 